#        将dataframe导入数据库


## 步骤
* json格式数据导入，python和java两表合并，去重
* 清洗时间、薪水
* 连接数据库
* 依次导入城市，公司和职位表
* 关闭连接

In [ ]:
import pandas as pd
import numpy as np
import pymysql
import json
import re
from pandas import DataFrame,Series

#读取josn格式文件
with open('C:\\Users\\jinbin\\Desktop\\job_全国_python_2018_03_29.json','r') as f:
    temp=json.loads(f.read())    
data1=DataFrame(temp)             #导入df
with open('C:\\Users\\jinbin\\Desktop\\job_全国_java_2018_03_29.json','r') as f:
    temp=json.loads(f.read())
data2=DataFrame(temp)


data1=data1.drop_duplicates()     #先去重后再合并
data2=data2.drop_duplicates()   
data=pd.concat([data1,data2],ignore_index=True)#合并java和python的数据
    


#清洗时间
time=[]                          
pat1='[_ ]|-'
for i in range(len(data['create_time'])):
    da=re.split(pat1,data['create_time'][i])
    str1=da[0] + "-" + da[1] + "-" + da[2]
    time.append(str1)
#去除原时间数据，并加入新的时间数据
data=data.drop('create_time',axis=1)
data=pd.merge(data,DataFrame(time,columns=['create_time']),right_index=True,left_index=True)


#清洗薪水
pat2=r'([kK]-)|[kK]'
smin=[]
smax=[]
for i in range(len(data['salary'])):
    str2=re.split(pat2,data['salary'][i])
    smin.append(str2[0])
    smax.append(str2[2])
#对有异常的最大薪水进行处理
for i in range(len(smax)):
    if (smax[i]=='以上'):
        smax[i]=smin[i]
#去除原薪水数据，并加入新的薪水数据
salary=DataFrame({'min_salary':smin,'max_salary':smax},columns=['min_salary','max_salary'])
data=data.drop('salary',axis=1)
data=pd.merge(data,salary,right_index=True,left_index=True)





#写入城市表
city_name = data['city'].value_counts().index #过滤掉重复的城市
city_id = np.arange(len(city_name)) + 1001    #赋予城市编号
city_dict = dict(zip(city_name,city_id))         #城市与编号对应的键值对
#连接数据库
conn = pymysql.connect(host = 'localhost',port = 3306,user = 'root',
                       passwd = 'apples',db = 'python', charset = "utf8")
cursor = conn.cursor()                   #创建游标对象
for i in range(len(city_name)):
    sql = "insert into city value" + str(tuple([city_id[i],city_name[i]]))
    cursor.execute(sql)
    conn.commit()
    
#写入公司表
cond = data.duplicated(['company_full_names'])   #得到公司名的出现重复时的位置   为True时表示此位置的值重复了
comp_full_name = data[-cond]['company_full_names']        #过滤掉重复的公司
comp_name = data[-cond]['company_name']                   #公司别名
comp_id = np.arange(len(comp_full_name)) + 2001                     #赋予公司编号
comp_dict = dict(zip(comp_full_name,comp_id))             #公司与编号对应的键值对
#循环sql语句导入数据
for i in range(len(comp_full_name)):
    sql = "insert into comp(comp_id,comp_full_name,comp_name) value" + str(tuple([comp_id[i],comp_full_name.iloc[i],comp_name.iloc[i]]))
    cursor.execute(sql)
    conn.commit()
    
#写入职位表
#有replace函数结合之间的城市编号  公司编号的键值对，替换成城市和公司的编号
DATA=pd.concat([data.drop('city',axis=1),data['city'].replace(city_dict)],axis=1)
DATA=pd.concat([DATA.drop('company_full_names',axis=1),
                DATA['company_full_names'].replace(comp_dict)],axis=1)

for i in range(len(DATA)):
    sql = '''insert into postn(postn_id,postn_name,postn_salary_max,postn_salary_min,postn_create_time,comp_id,city_id) value''' + str(tuple([DATA['positionId'][i],DATA['positionName'][i],DATA['max_salary'][i],DATA['min_salary'][i],DATA['create_time'][i],DATA['company_full_names'][i],DATA['city'][i]]))
    cursor.execute(sql)
    conn.commit()
    
cursor.close()    #关闭游标
conn.close()      #关闭连接